In [ ]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase


In [ ]:

load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo-0125"
temperature = 0.1
max_tokens = 1000



llm = ChatOpenAI(
    model = model_name,
    temperature=temperature,
    max_tokens=max_tokens,
    openai_api_key = openai_api_key
)

In [ ]:
db_file_path = "./chinook.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_file_path}")

print(db.dialect)
print(db.get_usable_table_names())


In [ ]:
query = "SELECT * FROM Artist LIMIT 10;"

db.run(query)


In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

tools = toolkit.get_tools()

tools


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage

SQL_PREFIX="""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.
"""


In [ ]:
from langchain_core.tools import tool
from langchain_community.utilities import GoogleSerperAPIWrapper


In [ ]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplies two integers together"""
    
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    """Sums two integers together"""
    
    return first_int + second_int

@tool
def exponentize(first_int: int, second_int: int) -> int:
    """Exponentize the base to the exponent value"""
    
    return first_int ** second_int

@tool("GoogleSearch")
def search(query_string: str):
    """
    Useful to search for any kinds of information and
    when you need to search the internet for any kinds of information, use this tool.
    """
    
    search = GoogleSerperAPIWrapper()
    
    return search.run(query_string)
    

In [ ]:
additional_tools = [add, multiply, exponentize, search]

final_tools = tools + additional_tools

In [ ]:
final_tools

In [ ]:
%pip install -qU langgraph

In [ ]:
system_message = SystemMessage(content=SQL_PREFIX)

agent_executor = create_react_agent(
    llm,
    tools = final_tools,
    state_modifier = system_message,
    verbose = True
)

In [ ]:
query = """
take 3 to the fifth power and multiply that result by the sum of twelve and three, then square the whole result, 
and also let me know the capital of Malaysia ...
and also need 5 bulleted points about Mahatma Gandhi Achievements
and Which country's customers have spent the most?
"""

In [ ]:
response = agent_executor.invoke({
    "messages": [
        HumanMessage(content=query)
    ]
})

response

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content=query)
    ]
}):
    print(stream)
    print("***********")
    